In [2]:
import pandas as pd
import sys, os

sys.path.append(os.path.abspath(os.path.join('../utils')))
from data_processing import DataProcessor

In [3]:
import warnings
import pandas as pd
import sys, os


from causalnex.structure import StructureModel


warnings.filterwarnings("ignore")

In [4]:
data_processer=DataProcessor()

In [20]:

file_path = '../data/processed/requests_df.csv'

# Read the CSV file into a DataFrame
order_df = pd.read_csv(file_path)

In [21]:
file_path = '../data/processed/clean_trip_weather.csv'

# Read the CSV file into a DataFrame
trip_df = pd.read_csv(file_path)

In [ ]:
order_df.info()

In [ ]:
trip_df.info()

## Merge the two datasets

In [24]:
# Merge based on status of the trip

driver_lat_ls = []
driver_lng_ls = []

for i, df in trip_df.iterrows():
    try:
        trip_id = df['order_id']
        drivers = order_df[order_df['order_id']==trip_id]
        driver = drivers[drivers['driver_action']=="accepted"]
        if len(driver)>0:
            driver_lat_ls.append(float(driver['lat']))
            driver_lng_ls.append(float(driver['lng']))
        else:
            if len(drivers) > 0:
                driver_lat_ls.append(0.5)
                driver_lng_ls.append(0.5)
            else:
                driver_lat_ls.append(0.0)
                driver_lng_ls.append(0.0)
    except Exception as e:
        driver_lat_ls.append(0.0)
        driver_lng_ls.append(0.0)

# create new columns in df_trip with the extracted driver locations
trip_df['driver_lat'] = driver_lat_ls
trip_df['driver_lng'] = driver_lng_ls

In [ ]:
trip_df

In [25]:
# trip_df = trip_df.drop('Unnamed: 0', axis=1)
# trip_df

,order_id,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,is_weekend,is_holiday,distance,speed,date,Rain_Condition,Temperature,driver_lat,driver_lng
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,0,0,16.123451,624.133590,2021-07-01,Rainy,High,0.000000,0.000000
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,0,0,1.884305,3.845521,2021-07-01,Rainy,High,0.000000,0.000000
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,0,0,13.822393,20.056676,2021-07-01,Rainy,High,0.000000,0.000000
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,0,0,1.511034,6.674507,2021-07-01,Rainy,High,0.000000,0.000000
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High,6.602207,3.270465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528415,1637696,"6.448218499999999,3.4772075","6.437787399999999,3.481670199999999",2021-12-30 20:35:06,2021-12-30 21:02:59,0,0,1.254752,2.700005,2021-12-30,Not Rainy,High,0.000000,0.000000
528416,1637702,"6.442320899999999,3.4736868","6.436589333407897,3.5559738188407835",2021-12-30 20:48:13,2021-12-30 21:43:49,0,0,9.124781,9.846886,2021-12-30,Not Rainy,High,0.000000,0.000000
528417,1637704,"6.4281982,3.492248","6.448088500000001,3.4775747",2021-12-30 20:51:45,2021-12-30 21:41:32,0,0,2.733704,3.294722,2021-12-30,Not Rainy,High,0.000000,0.000000
528418,1637705,"6.5869296,3.3632966","6.637906899999999,3.3339515",2021-12-30 20:48:50,2021-12-30 21:08:28,0,0,6.504796,19.878833,2021-12-30,Not Rainy,High,0.000000,0.000000


In [26]:
driver_lat_coordinates = trip_df['driver_lat'].tolist()
driver_lng_coordinates = trip_df['driver_lng'].tolist()
ending_coordinates = []
for i in range(len(driver_lat_coordinates)):
    ending_coordinates.append(f'{driver_lat_coordinates[i]},{driver_lng_coordinates[i]}')

In [27]:
start_coordinates = trip_df['Trip Origin'].tolist()


In [28]:
trip_df['driver_distance'] = data_processer.calculate_distances(start_coordinates, ending_coordinates)


## Driver distance distance calculation

In [34]:
# trip_df

In [33]:
# trip_df['driver_distance'].value_counts()

## label-based selection or boolean indexing

In [39]:
trip_df

,order_id,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,is_weekend,is_holiday,distance,speed,date,Rain_Condition,Temperature,driver_lat,driver_lng,driver_distance,is_fulfilled
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,0,0,16.123451,624.133590,2021-07-01,Rainy,High,0.000000,0.000000,-1.000000,1
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,0,0,1.884305,3.845521,2021-07-01,Rainy,High,0.000000,0.000000,-1.000000,1
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,0,0,13.822393,20.056676,2021-07-01,Rainy,High,0.000000,0.000000,-1.000000,1
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,0,0,1.511034,6.674507,2021-07-01,Rainy,High,0.000000,0.000000,-1.000000,1
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,0,0,20.984319,348.126952,2021-07-01,Rainy,High,6.602207,3.270465,0.694264,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528415,1637696,"6.448218499999999,3.4772075","6.437787399999999,3.481670199999999",2021-12-30 20:35:06,2021-12-30 21:02:59,0,0,1.254752,2.700005,2021-12-30,Not Rainy,High,0.000000,0.000000,-1.000000,1
528416,1637702,"6.442320899999999,3.4736868","6.436589333407897,3.5559738188407835",2021-12-30 20:48:13,2021-12-30 21:43:49,0,0,9.124781,9.846886,2021-12-30,Not Rainy,High,0.000000,0.000000,-1.000000,1
528417,1637704,"6.4281982,3.492248","6.448088500000001,3.4775747",2021-12-30 20:51:45,2021-12-30 21:41:32,0,0,2.733704,3.294722,2021-12-30,Not Rainy,High,0.000000,0.000000,-1.000000,1
528418,1637705,"6.5869296,3.3632966","6.637906899999999,3.3339515",2021-12-30 20:48:50,2021-12-30 21:08:28,0,0,6.504796,19.878833,2021-12-30,Not Rainy,High,0.000000,0.000000,-1.000000,1


In [31]:
trip_with_drivers_df= trip_df[trip_df['driver_distance']!=-1]
trip_with_drivers_df.shape

(26176, 15)

In [35]:
trip_df.to_csv("../data/features/trip_with_driver_distance.csv")

## Extract the unfullfilled orders

In [48]:
trip_with_drivers_df['is_fulfilled'] = trip_df['driver_distance'].apply(
    lambda x: 1 if x !=-2 else 0)

In [49]:
trip_with_drivers_df[trip_with_drivers_df['is_fulfilled']==0].shape

(605, 16)

## Filter out the neccessary columns only

In [50]:
filtered_df = trip_with_drivers_df.loc[:, ['is_weekend', 'is_holiday', 'distance', 'speed', 'Rain_Condition', 'Temperature', 'driver_distance', 'is_fulfilled']]
filtered_df

,is_weekend,is_holiday,distance,speed,Rain_Condition,Temperature,driver_distance,is_fulfilled
4,0,0,20.984319,348.126952,Rainy,High,0.694264,1
5,0,0,8.127195,14.237422,Rainy,High,1.959786,1
6,0,0,5.040921,4.929996,Rainy,High,2.947683,1
7,0,0,5.609962,9.607927,Rainy,High,1.197389,1
8,0,0,19.082399,25.854963,Rainy,High,0.197853,1
...,...,...,...,...,...,...,...,...
44373,0,0,2.582307,3.661404,Not Rainy,High,1.633417,1
44395,1,0,3.590521,22.209406,Rainy,High,2.736381,1
46224,1,0,26.751642,33.661626,Rainy,Moderate,3.094992,1
50522,0,0,18.333328,16.450643,Not Rainy,High,1.842496,1


In [51]:
filtered_df['is_fulfilled'].value_counts()

1    25571
0      605
Name: is_fulfilled, dtype: int64

## Create Structure Model

In [52]:
sm = StructureModel()


# Create Edges

In [60]:
#  Transform and encode

import numpy as np

struct_data = filtered_df.copy()
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)

print(non_numeric_columns)



['Rain_Condition', 'Temperature']


In [61]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

struct_data.head(5)

,is_weekend,is_holiday,distance,speed,Rain_Condition,Temperature,driver_distance,is_fulfilled
4,0,0,20.984319,348.126952,1,0,0.694264,1
5,0,0,8.127195,14.237422,1,0,1.959786,1
6,0,0,5.040921,4.929996,1,0,2.947683,1
7,0,0,5.609962,9.607927,1,0,1.197389,1
8,0,0,19.082399,25.854963,1,0,0.197853,1


In [63]:
# Non numeric 
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)

print(non_numeric_columns)

[]


In [ ]:
# struct_data['Rain_Condition'].value_counts()

### Scale and normalize numerical data

In [64]:
# Scale and normalize numerical data
from sklearn import preprocessing

x = struct_data.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)


In [99]:
df

,0,1,2,3,4,5,6,7
0,0.0,0.0,0.614757,0.061171,1.0,0.0,0.076714,1.0
1,0.0,0.0,0.237800,0.002499,1.0,0.0,0.112747,1.0
2,0.0,0.0,0.147314,0.000863,1.0,0.0,0.140875,1.0
3,0.0,0.0,0.163998,0.001685,1.0,0.0,0.091039,1.0
4,0.0,0.0,0.558995,0.004540,1.0,0.0,0.062579,1.0
...,...,...,...,...,...,...,...,...
26171,0.0,0.0,0.075230,0.000640,0.0,0.0,0.103454,1.0
26172,1.0,0.0,0.104790,0.003900,1.0,0.0,0.134859,1.0
26173,1.0,0.0,0.783849,0.005912,1.0,1.0,0.145069,1.0
26174,0.0,0.0,0.537033,0.002888,0.0,0.0,0.109407,1.0


In [65]:
# Apply the NOTEARS algorithm to learn the structure

from causalnex.structure.notears import from_pandas

sm = from_pandas(df)

In [ ]:

from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)

viz.toggle_physics(False)
viz.show("../screenshots/01_fully_connected.html")

In [ ]:
# import graphviz

# # Create the Graphviz graph
# graph = graphviz.Digraph()

# # Add nodes and edges to the graph
# for node in [str(n) for n in sm.nodes]:
#     graph.node(node)

# for (u, v) in sm.edges:
#     graph.edge(str(u), str(v))

# # Display the graph in the default web browser
# graph.view()


### Apply threshold to weaker edges

In [ ]:
sm.remove_edges_below_threshold(0.8)

viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)

viz.toggle_physics(False)
viz.show("../screenshots/02_fully_connected.html")

In [107]:
df = df.rename(columns={0: "is_weekend", 1: "is_holiday", 2: "distance", 3: "speed",4: "Rain_Condition", 5: "Temperature",6:"driver_distance,", 7:"is_fulfilled"}, errors="raise")


In [103]:
df

,is_weekend,is_holiday,distance,speed,Rain_Condition,Temperature,"driver_distance,",is_fulfilled
0,0.0,0.0,0.614757,0.061171,1.0,0.0,0.076714,1.0
1,0.0,0.0,0.237800,0.002499,1.0,0.0,0.112747,1.0
2,0.0,0.0,0.147314,0.000863,1.0,0.0,0.140875,1.0
3,0.0,0.0,0.163998,0.001685,1.0,0.0,0.091039,1.0
4,0.0,0.0,0.558995,0.004540,1.0,0.0,0.062579,1.0
...,...,...,...,...,...,...,...,...
26171,0.0,0.0,0.075230,0.000640,0.0,0.0,0.103454,1.0
26172,1.0,0.0,0.104790,0.003900,1.0,0.0,0.134859,1.0
26173,1.0,0.0,0.783849,0.005912,1.0,1.0,0.145069,1.0
26174,0.0,0.0,0.537033,0.002888,0.0,0.0,0.109407,1.0


In [108]:
sm = from_pandas(df, tabu_edges=[("driver_distance,", "is_weekend"),("distance", "is_weekend")], w_threshold=0.8)


In [ ]:

sm.remove_edges_below_threshold(0.8)

viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)

viz.toggle_physics(False)
viz.show("../screenshots/03_fully_connected.html")